In [1]:
# import all relevant libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import requests
import json
from sec_api import QueryApi



I am using base code from https://gist.github.com/janlukasschroeder/3da274150fd00c1c1776c7e541a9b61b to get me started in scraping financial statements from the SEC. I will be using 10-K forms (annual SEC filing) rather than all quarterly filings (10-Q) to reduce the size of data.

In [15]:
from sec_api import QueryApi

api_key = '88140c3a554da7220ba7453a5e1ea2221c5f1e0e42c3293bbe75857245774e2e'

# get your API key at https://sec-api.io
query_api = QueryApi(api_key=api_key)

# fetch all 10-K filings for Apple from the last 25 years to test this API
query = {
    "query": {
        "query_string": {
            "query": "filedAt:[2020-01-01 TO 2020-12-31] AND (formType:\"10-K\") AND ticker:AAPL"
        }
    },
    "from": "0",
    "size": "25",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

query_result = query_api.get_filings(query)

In [10]:
# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

In [16]:
query_result

{'total': {'value': 1, 'relation': 'eq'},
 'query': {'from': 0, 'size': 25},
 'filings': [{'ticker': 'AAPL',
   'formType': '10-K',
   'accessionNo': '0000320193-20-000096',
   'cik': '320193',
   'companyNameLong': 'Apple Inc. (Filer)',
   'companyName': 'Apple Inc.',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926.htm',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/0000320193-20-000096.txt',
   'filedAt': '2020-10-29T18:06:25-04:00',
   'documentFormatFiles': [{'sequence': '1',
     'size': '2467306',
     'documentUrl': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/320193/000032019320000096/aapl-20200926.htm',
     'description': '10-K',
     'type': '10-K'},
    {'sequence': '2',
     'size': '123356',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/a10-kexhib

In [5]:
sample_company = query_result['filings'][0]

In [7]:
acc_no = sample_company['accessionNo']
acc_no

'0001640334-22-001688'

In [13]:
accession_numbers = []

# extract accession numbers of each filing
for filing in query_result['filings']:
    accession_numbers.append(filing['accessionNo']);

accession_numbers

['0000320193-21-000105',
 '0000320193-20-000096',
 '0000320193-19-000119',
 '0000320193-18-000145',
 '0000320193-17-000070',
 '0001628280-16-020309',
 '0001193125-15-356351',
 '0001193125-14-383437',
 '0001193125-13-416534',
 '0001193125-12-444068',
 '0001193125-11-282113',
 '0001193125-10-238044',
 '0001193125-10-012091',
 '0001193125-09-214859',
 '0001193125-08-224958',
 '0001047469-07-009340',
 '0001104659-06-084288',
 '0001104659-06-081617',
 '0001104659-05-058421',
 '0001047469-04-035975',
 '0001047469-03-041604',
 '0001047469-02-007674',
 '0000912057-00-053623',
 '0000912057-99-010244',
 '0001047469-98-001822']

In [8]:
import time

# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no, retry = 0):
    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + api_key

    # linear backoff in case API fails with "too many requests" error
    try:
      response_tmp = requests.get(request_url)
      xbrl_json = json.loads(response_tmp.text)
    except:
      if retry > 5:
        raise Exception('API error')
      
      # wait 500 milliseconds on error and retry
      time.sleep(0.5) 
      return get_xbrl_json(accession_no, retry + 1)

    return xbrl_json

In [13]:
sample_xbrl = get_xbrl_json(acc_no)

In [14]:
sample_xbrl

{'status': 500, 'error': 'request failed'}

In [3]:
# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 



In [54]:
example_income_statement = get_income_statement(get_xbrl_json(accession_numbers[10]))

In [58]:
example_income_one = example_income_statement.iloc[:, -4]

In [59]:
pd.DataFrame.to_csv(example_income_one, './data/apple_2011_income.csv')

In [17]:
loaded_income = pd.read_csv('./data/apple_2011_income.csv')

In [23]:
loaded_income.T


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Unnamed: 0,CostOfGoodsAndServicesSold,EarningsPerShareBasic,EarningsPerShareDiluted,GrossProfit,IncomeLossFromContinuingOperationsBeforeIncome...,IncomeTaxExpenseBenefit,NetIncomeLoss,NonoperatingIncomeExpense,OperatingExpenses,OperatingIncomeLoss,ResearchAndDevelopmentExpense,SalesRevenueNet,SellingGeneralAndAdministrativeExpense,WeightedAverageNumberOfDilutedSharesOutstanding,WeightedAverageNumberOfSharesOutstandingBasic
2010-09-26-2011-09-24,64431000000.0,28.05,27.68,43818000000.0,34205000000.0,8283000000.0,25922000000.0,415000000.0,10028000000.0,33790000000.0,2429000000.0,108249000000.0,7599000000.0,936645000.0,924258000.0


In [5]:
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}

    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 


    cash_flows = pd.DataFrame(cash_flows_store)
    return cash_flows.T

In [6]:
example_cash_flow = get_cash_flow_statement(get_xbrl_json(accession_numbers[10]))

NameError: name 'accession_numbers' is not defined

In [ ]:
example_cash_flow_one = example_cash_flow.iloc[:, -5]

In [52]:
pd.DataFrame.to_csv(example_cash_flow_one, './data/apple_2011_cashflow.csv')

In [19]:
loaded_cash_flow = pd.read_csv('./data/apple_2011_cashflow.csv')

In [20]:
loaded_cash_flow

,Unnamed: 0,2010-09-26-2011-09-24
0,CashAndCashEquivalentsAtCarryingValue,NaN
1,CashAndCashEquivalentsPeriodIncreaseDecrease,-1.446000e+09
2,DeferredIncomeTaxExpenseBenefit,2.868000e+09
3,DepreciationAmortizationAndAccretionNet,1.814000e+09
4,EmployeeServiceShareBasedCompensationCashUsedT...,5.200000e+08
5,ExcessTaxBenefitFromShareBasedCompensationFina...,1.133000e+09
6,IncomeTaxesPaidNet,3.338000e+09
7,IncreaseDecreaseInAccountsPayable,2.515000e+09
8,IncreaseDecreaseInAccountsReceivable,-1.430000e+08
9,IncreaseDecreaseInDeferredRevenue,1.654000e+09


In [8]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet.T

In [26]:
example_balance = get_balance_sheet(get_xbrl_json(accession_numbers[10]))

In [45]:
example_balance_T = pd.DataFrame(example_balance.iloc[:, -1]).T

In [53]:
pd.DataFrame.to_csv(example_balance_T, './data/apple_2011_balance.csv')

In [21]:
loaded_balance = pd.read_csv('./data/apple_2011_balance.csv')

In [22]:
loaded_balance.head()

,Unnamed: 0,AccountsPayableCurrent,AccountsReceivableNetCurrent,AccruedLiabilitiesCurrent,AccumulatedOtherComprehensiveIncomeLossNetOfTax,Assets,AssetsCurrent,AvailableForSaleSecuritiesCurrent,AvailableForSaleSecuritiesNoncurrent,CashAndCashEquivalentsAtCarryingValue,...,Liabilities,LiabilitiesAndStockholdersEquity,LiabilitiesCurrent,NontradeReceivablesCurrent,OtherAssetsCurrent,OtherAssetsNoncurrent,OtherLiabilitiesNoncurrent,PropertyPlantAndEquipmentAndCapitalizedSoftwareNet,RetainedEarningsAccumulatedDeficit,StockholdersEquity
0,2011-09-24,14632000000,5369000000,9247000000,443000000,116371000000,44988000000,16137000000,55618000000,9815000000,...,39756000000,116371000000,27970000000,6348000000,4529000000,3556000000,10100000000,7777000000,62841000000,76615000000


In [11]:
# clean income statement.
# drop duplicate columns (= column name ends with "_left"), drop key_0 column, drop columns with +5 NaNs
def clean_income_statement(statement):
    for column in statement:

        # column has more than 5 NaN values
        is_nan_column = statement[column].isna().sum() > 5

        if column.endswith('_left') or column == 'key_0' or is_nan_column:
            statement = statement.drop(column, axis=1)
    
    # rearrange columns so that first column represents first quarter
    # e.g. 2018, 2019, 2020 - and not 2020, 2019, 2018
    sorted_columns = sorted(statement.columns.values)
    
    return statement[sorted_columns]

In [12]:
# merge two income statements into one statement.
# row indicies of both statements have to be the same
# statement_b represents the most recent statement.
def merge_income_statements(statement_a, statement_b):
    return statement_a.merge(statement_b,
                     how="outer", 
                    #  on=statement_b.index, 
                    right_on=statement_b.index, 
                     left_index=True,
                    #  right_index=True,
                     suffixes=('_left', ''))

In [19]:
# helps printing dataframes while we generate new income statement
from IPython.display import display, HTML

previous_income_statement_set = False
income_statement_final = None

# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

for accession_no in accession_numbers[0:5]:
# for accession_no in accession_numbers: # doesn't work with filings filed before 2017 - indicies not equal
    print('Processing: ' + accession_no)
    
    # get XBRL-JSON of 10-Q or 10-K filing by accession number
    xbrl_json_data = get_xbrl_json(accession_no)
    
    # convert XBRL-JSON to a pandas dataframe
    income_statement_uncleaned = get_income_statement(xbrl_json_data)

    # clean the income statement
    income_statement_cleaned = clean_income_statement(income_statement_uncleaned)
    
    # print income statement on each iteration to monitor progress
    # display(HTML(income_statement_cleaned.to_html()))
    
    # merge new income statement with previously generated income statement
    if previous_income_statement_set:
        income_statement_final = clean_income_statement(merge_income_statements(income_statement_final, income_statement_cleaned))
    else:
        income_statement_final = income_statement_cleaned
        previous_income_statement_set = True

Processing: 0000320193-21-000105
Processing: 0000320193-20-000096
Processing: 0000320193-19-000119
Processing: 0000320193-18-000145
Processing: 0000320193-17-000070


In [21]:
income_statement_final.T

,NaN,CostOfGoodsAndServicesSold,GrossProfit,ResearchAndDevelopmentExpense,SellingGeneralAndAdministrativeExpense,OperatingExpenses,OperatingIncomeLoss,NonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,NetIncomeLoss,EarningsPerShareBasic,EarningsPerShareDiluted,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfDilutedSharesOutstanding,NaN,SalesRevenueNet,CommonStockDividendsPerShareDeclared
2014-09-28-2015-09-26,NaN,140089000000,93626000000,8067000000,14329000000,22396000000,71230000000,1285000000,72515000000,19121000000,53394000000,9.28,9.22,5753421000,5793069000,NaN,233715000000,1.98
2015-09-27-2016-09-24,NaN,131376000000,84263000000,10045000000,14194000000,24239000000,60024000000,1348000000,61372000000,15685000000,45687000000,8.35,8.31,5470820000,5500281000,NaN,215639000000,2.18
2016-09-25-2017-09-30,NaN,141048000000,88186000000,11581000000,15261000000,26842000000,61344000000,2745000000,64089000000,15738000000,48351000000,9.27,9.21,5217242000,5251692000,NaN,229234000000,2.4
2017-10-01-2018-09-29,NaN,163756000000,101839000000,14236000000,16705000000,30941000000,70898000000,2005000000,72903000000,13372000000,59531000000,12.01,11.91,4955377000,5000109000,265595000000,NaN,NaN
2018-09-30-2019-09-28,260174000000,161782000000,98392000000,16217000000,18245000000,34462000000,63930000000,1807000000,65737000000,10481000000,55256000000,11.97,11.89,4617834000,4648913000,NaN,NaN,NaN
2019-09-29-2020-09-26,274515000000,169559000000,104956000000,18752000000,19916000000,38668000000,66288000000,803000000,67091000000,9680000000,57411000000,3.31,3.28,17352119000,17528214000,NaN,NaN,NaN
2020-09-27-2021-09-25,365817000000,212981000000,152836000000,21914000000,21973000000,43887000000,108949000000,258000000,109207000000,14527000000,94680000000,5.67,5.61,16701272000,16864919000,NaN,NaN,NaN
